# Tech companies in California

## 1º Import the libraries 

In [1]:
from pymongo import MongoClient
import pandas as pd

## 2º Conection with the client.

In [2]:
client = MongoClient()

## 3º Conection with database in Mongo.

In [3]:
db = client.ironhack
db.companies

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'companies')

## 4º All tech companies in California that raising more than 1 million.

In [4]:
filt = {"$and":[{"offices.state_code":{"$eq":"CA"}},{"offices.country_code":{"$eq":"USA"}},{"total_money_raised":{"$regex":"M"}},{"number_of_employees":{"$lte":200}},{"category_code":{"$gte":"software"}}]}
proyect = {"_id":0,"name":1,"total_money_raised":1,"offices.city":1,"offices.state_code":1,"offices.country_code":1,"category_code":1,"founded_year":1,"number_of_employees":1}

result = db.companies.find(filt,proyect).sort("founded_year",-1)

tech_comp_cali = list(result)
#tech_comp_cali

In [5]:
df_tech_comp_cali = pd.json_normalize(tech_comp_cali,"offices",["name","category_code","number_of_employees","founded_year","total_money_raised"])
#df_tech_comp_cali

In [19]:
#Clean the DataSet.
df_tech_comp_cali  = df_tech_comp_cali.drop_duplicates()
df_tech_comp_cali = df_tech_comp_cali[df_tech_comp_cali["state_code"] == "CA"]
df_tech_comp_cali = df_tech_comp_cali[df_tech_comp_cali["country_code"] == "USA"]
#df_tech_comp_cali

In [7]:
f"All tech companies in California that raising more than 1 million : {len(df_tech_comp_cali.index)}"

'All tech companies in California that raising more than 1 million : 217'

## 5º All tech companies in California that raise more than 1 million and were founded starting in 2009.

In [8]:
filt = {"$and":[{"offices.state_code":{"$eq":"CA"}},{"offices.country_code":{"$eq":"USA"}},{"total_money_raised":{"$regex":"M"}},{"number_of_employees":{"$lte":200}},{"category_code":{"$gte":"software"}},{"founded_year":{"$gte":2009}}]}
proyect = {"_id":0,"name":1,"total_money_raised":1,"offices.city":1,"offices.state_code":1,"offices.country_code":1,"offices.latitude":1,"offices.longitude":1,"category_code":1,"founded_year":1,"number_of_employees":1}

result = db.companies.find(filt,proyect).sort("founded_year",-1)
list_companies = list(result)

print("All tech companies in California that raise more than 1 million and were founded starting in 2009 : ",len(list_companies))
#list_companies

All tech companies in California that raise more than 1 million and were founded starting in 2009 :  11


In [9]:
df_new_office = pd.json_normalize(list_companies,"offices",["name","category_code","number_of_employees","founded_year","total_money_raised"])
#df_new_office

In [10]:
#Clean the DataSet.

df_new_office = df_new_office.dropna(axis=0, how="any")
df_new_office = df_new_office.drop_duplicates()
df_new_office = df_new_office[df_new_office["state_code"] == "CA"]
df_new_office = df_new_office[df_new_office["country_code"] == "USA"]

In [11]:
df_new_office

,city,state_code,country_code,latitude,longitude,name,category_code,number_of_employees,founded_year,total_money_raised
0,San Mateo,CA,USA,37.566879,-122.323895,Fixya,web,30,2013,$8M
11,San Francisco,CA,USA,39.905226,-86.054702,Formspring,web,19,2009,$14.3M
12,Santa Monica,CA,USA,34.007112,-118.489748,Tongal,web,8,2009,$16M
14,Menlo Park,CA,USA,37.448491,-122.180281,Factery,web,2,2009,$1.2M


# 6º Map with the companies.

In [13]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

In [14]:
inicial_lat = 37.566879
inicial_long = -122.323895

In [15]:
maps = folium.Map(location = [inicial_lat,inicial_long], zoom_start=15)

In [16]:
San_Mateo = Marker(location = [inicial_lat,inicial_long],tooltip="New office")
San_Mateo.add_to(maps)

In [18]:
for i,row in df_new_office.iterrows():
    offices = {
        "location" : [row["latitude"],row["longitude"]],
        "tooltip" : row["name"]}
    if row["latitude"] == 37.56289 and row["longitude"]== -122.324005 :
         icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "coffee",
                    icon_color = "black")

    Marker(**offices).add_to(maps)
maps


# 7º New office in San Mateo.

In [23]:
df_new_office.head(1)

,city,state_code,country_code,latitude,longitude,name,category_code,number_of_employees,founded_year,total_money_raised
0,San Mateo,CA,USA,37.566879,-122.323895,Fixya,web,30,2013,$8M
